In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("~/Downloads/moloco.csv")
df.head(4)

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG


In [3]:
df.describe()

,ts,user_id,country_id,site_id
count,3553,3553,3553,3553
unique,3538,1916,7,8
top,2019-02-03 20:00:34,LC3A59,TL6,5NPAU
freq,3,149,1449,1614


In [4]:
df.shape

(3553, 4)

### Q1.1 Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

In [5]:
df_bdv = df[df.country_id == 'BDV']
df_bdv.shape

(844, 4)

In [6]:
df_bdv_grp = df_bdv.groupby('site_id').user_id.nunique().to_frame() 
df_bdv_grp.sort_values('user_id',ascending = False)

,user_id
site_id,
5NPAU,544
N0OTG,90
3POLC,2


### Q1.2 Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [7]:
sub_df = df[df.ts > '2019-02-03 00:00:00']
sub_df = sub_df[sub_df.ts < '2019-02-04 23:59:5']
sub_df.groupby(['site_id','user_id']).user_id.count().reset_index(name='count') \
                             .sort_values(['count'], ascending=False).head(4)

,site_id,user_id,count
557,N0OTG,LC3A59,26
407,N0OTG,LC06C3,25
578,N0OTG,LC3C9D,17
3,3POLC,LC3C7E,15


### Q1.3 For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

In [8]:
df_last_ts = df.groupby(['user_id']).ts.max().reset_index(name='ts')
df_last_ts.shape

(1916, 2)

In [9]:
df_last_ts_updated = pd.merge(df, df_last_ts, on=['user_id','ts'], how='inner')
df_last_ts_updated.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
1,2019-02-01 00:21:50,LC3500,TL6,N0OTG
2,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G
3,2019-02-01 00:41:50,LCC3C3,QLT,5NPAU
4,2019-02-01 00:42:13,LC39C8,TL6,QGO3G


In [10]:
df_last_ts_updated.groupby('site_id').user_id.nunique().reset_index(name='uni_count').\
                sort_values('uni_count', ascending = False)

,site_id,uni_count
1,5NPAU,992
5,N0OTG,561
6,QGO3G,289
3,GVOFK,42
0,3POLC,28
7,RT9Z6,2
2,EUZ/Q,1
4,JSUUP,1


### Q1.4 For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [11]:
df_first_ts = df.groupby(['user_id']).ts.min().reset_index(name='ts')
df_last_ts = df.groupby(['user_id']).ts.max().reset_index(name='ts')
df_first_ts_updated = pd.merge(df[['user_id','ts', 'site_id']], df_first_ts, on=['user_id','ts'], how='inner')
df_last_ts_updated = pd.merge(df[['user_id','ts', 'site_id']], df_last_ts, on=['user_id','ts'], how='inner')

df_merge_final = pd.merge(df_first_ts_updated, df_last_ts_updated, on=['user_id'], how='inner')
df_merge_final = df_merge_final[df_merge_final.site_id_x == df_merge_final.site_id_y]
len(df_merge_final)

1670